In [2]:
#Andrea Burns
#Data Reader
import pandas as pd
import numpy as np
import itertools

In [30]:
#read each data frame in from my excel sheets
xls = pd.ExcelFile('DataDownload.xls')
df1 = pd.read_excel(xls, 'ACCESS')
df2 = pd.read_excel(xls, 'STORES')
df3 = pd.read_excel(xls, 'RESTAURANTS')
df4 = pd.read_excel(xls, 'ASSISTANCE')
df5 = pd.read_excel(xls, 'INSECURITY')
df6 = pd.read_excel(xls, 'PRICES_TAXES')
df7 = pd.read_excel(xls, 'LOCAL')
df8 = pd.read_excel(xls, 'HEALTH')
df9 = pd.read_excel(xls, 'SOCIOECONOMIC')

In [31]:
#these will be my output space; I'll try to predict these features
diabetes08 = df8['PCT_DIABETES_ADULTS08'].as_matrix()
diabetes13 = df8['PCT_DIABETES_ADULTS13'].as_matrix()
obesity08 = df8['PCT_OBESE_ADULTS08'].as_matrix()
obesity13 = df8['PCT_OBESE_ADULTS13'].as_matrix()

In [32]:
#remove my Y from the feature space to train on
df8 = df8.drop('PCT_DIABETES_ADULTS08', 1)
df8 = df8.drop('PCT_DIABETES_ADULTS13', 1)
df8 = df8.drop('PCT_OBESE_ADULTS08', 1)
df8 = df8.drop('PCT_OBESE_ADULTS13', 1)

In [6]:
alldata = [df1,df2,df3,df4,df5,df6,df7,df8,df9] 

In [7]:
#formatting all instances
X = []
i = 0
for i in range(0,3143):
    current_row = []
    for frame in alldata:
        mat = frame.as_matrix()
        current_row.append(mat[i][3:])
    X.append(list(itertools.chain(*current_row)))

In [8]:
#some features are missing data for specific instances, so I want to find them and deal with them
missing = np.argwhere(np.isnan(X))

In [9]:
feats = []
for i in range(0,len(missing)):
    feats.append(missing[i][1])
print(set(feats))

{1, 2, 4, 6, 7, 9, 11, 12, 14, 15, 16, 18, 19, 21, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 43, 46, 49, 52, 55, 58, 61, 64, 67, 69, 70, 73, 76, 79, 82, 85, 88, 93, 94, 95, 99, 100, 101, 115, 116, 117, 118, 125, 126, 127, 128, 129, 130, 146, 147, 148, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 169, 170, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 254, 257, 266, 267, 269, 272}


In [10]:
#these are two of the ~10 categorical features. Only these two have missing features
#since they are 0 or 1 binary classes, it doesn't make sense to replace NaN values with the mean of the column
#so instead I replace it will -1 as a "missing" class
for i in range(0,len(missing)):
    if missing[i][1] == 249:
        X[missing[i][0]][249] = -1
    elif missing[i][1] == 250:
        X[missing[i][0]][250] = -1

In [11]:
#all other features are continuous so I can impute NaN values with the mean of the feature!
from sklearn.preprocessing import Imputer
# missing_values is the value of your placeholder, strategy is if you'd like mean, median or mode, and axis=0 means it calculates the imputation based on the other feature values for that sample
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(X)
inputspace = imp.transform(X)

In [12]:
#I didn't previously realize that the diabetes and obesity vectors had Nan 
#now fixing it similarly with the below cells, only a few were missing (3 or 4)
imp2 = Imputer(missing_values='NaN', strategy='mean', axis=0)
diabetes08=diabetes08.reshape(-1,1)
imp2.fit(diabetes08)
outputspace1 = imp2.transform(diabetes08)

In [13]:
imp3 = Imputer(missing_values='NaN', strategy='mean', axis=0)
diabetes13=diabetes13.reshape(-1,1)
imp3.fit(diabetes13)
outputspace2 = imp3.transform(diabetes13)

In [14]:
imp4 = Imputer(missing_values='NaN', strategy='mean', axis=0)
obesity08=obesity08.reshape(-1,1)
imp4.fit(obesity08)
outputspace3 = imp4.transform(obesity08)

In [15]:
imp5 = Imputer(missing_values='NaN', strategy='mean', axis=0)
obesity13=obesity13.reshape(-1,1)
imp5.fit(obesity13)
outputspace4 = imp5.transform(obesity13)

In [16]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as prep
inputspace = prep.scale(inputspace)

#making different classifiers to try to predict the four rates 
X_train, X_test, y_train, y_test = train_test_split(inputspace,outputspace1.ravel(), test_size=0.1, random_state=42)
diab8clf = SVR(C=1.0, epsilon=0.2)

X_train2, X_test2, y_train2, y_test2 = train_test_split(inputspace,outputspace2.ravel(), test_size=0.1, random_state=42)
diab13clf = SVR(C=1.0, epsilon=0.2)

X_train3, X_test3, y_train3, y_test3 = train_test_split(inputspace,outputspace3.ravel(), test_size=0.1, random_state=42)
obes8clf = SVR(C=1.0, epsilon=0.2)

X_train4, X_test4, y_train4, y_test4 = train_test_split(inputspace,outputspace4.ravel(), test_size=0.1, random_state=42)
obes13clf = SVR(C=1.0, epsilon=0.2)

In [119]:
#fitting train data with SVM model for regression, as I am predicting the diabetes or obesity rates
#which are continuous values
diab8clf.fit(X_train,y_train)
diab13clf.fit(X_train2,y_train2)
obes8clf.fit(X_train3,y_train3)
obes13clf.fit(X_train4,y_train4)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.2, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [120]:
#test portion 
diabetes8pred = diab8clf.predict(X_test)
diabetes13pred = diab13clf.predict(X_test2)
obesity8pred = obes8clf.predict(X_test3)
obesity13pred = obes13clf.predict(X_test4)

In [122]:
#these are two standard performance metrics for regression, printing them for each
#mean squared error has been shown in class, kind of like avg. distance btw output and target values
#pearson coefficient is a statistic used to show the strength of a linear relationship
#best pearson value possible is -1 or 1 (perfectly negative or positive relationship)

from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
print('Pearson Coefficient and MSE for Diabetes Prediction in 2008')
print(pearsonr(np.asarray(y_test).ravel(),diabetes8pred.ravel()))
mean_sqr1 = mean_squared_error(diabetes8pred, y_test)
print(mean_sqr1)
print()

print('Pearson Coefficient and MSE for Diabetes Prediction in 2013')
print(pearsonr(np.asarray(y_test2).ravel(),diabetes13pred.ravel()))
mean_sqr2 = mean_squared_error(diabetes13pred, y_test2)
print(mean_sqr2)
print()

print('Pearson Coefficient and MSE for Obesity Prediction in 2008')
print(pearsonr(np.asarray(y_test3).ravel(),obesity8pred.ravel()))
mean_sqr3 = mean_squared_error(obesity8pred, y_test3)
print(mean_sqr3)
print()

print('Pearson Coefficient and MSE for Obesity Prediction in 2013')
print(pearsonr(np.asarray(y_test4).ravel(),obesity13pred.ravel()))
mean_sqr4 = mean_squared_error(obesity13pred, y_test4)
print(mean_sqr4)

Pearson Coefficient and MSE for Diabetes Prediction in 2008
(0.9119756128617906, 3.7663255632562475e-123)
0.82236275945728

Pearson Coefficient and MSE for Diabetes Prediction in 2013
(0.8740210255469344, 3.960268806775074e-100)
1.5043250592260557

Pearson Coefficient and MSE for Obesity Prediction in 2008
(0.8363386149441886, 1.052059362694424e-83)
4.542912274967804

Pearson Coefficient and MSE for Obesity Prediction in 2013
(0.8165881401494428, 1.0994118503700413e-76)
7.158954486310911


In [18]:
from sklearn import linear_model
reg1 = linear_model.Lasso(alpha = 0.1)
reg1.fit(X_train,y_train)

reg2 = linear_model.Lasso(alpha = 0.1)
reg2.fit(X_train2,y_train2)

reg3 = linear_model.Lasso(alpha = 0.1)
reg3.fit(X_train3,y_train3)

reg4 = linear_model.Lasso(alpha = 0.1)
reg4.fit(X_train4,y_train4)

#test portion 
diabetes8predLasso = reg1.predict(X_test)
diabetes13predLasso = reg2.predict(X_test2)
obesity8predLasso = reg3.predict(X_test3)
obesity13predLasso = reg4.predict(X_test4)

from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
print('Pearson Coefficient and MSE for Diabetes Prediction in 2008 using Lasso')
print(pearsonr(np.asarray(y_test).ravel(),diabetes8predLasso.ravel()))
mean_sqr1 = mean_squared_error(diabetes8predLasso, y_test)
print(mean_sqr1)
print()

print('Pearson Coefficient and MSE for Diabetes Prediction in 2013 using Lasso')
print(pearsonr(np.asarray(y_test2).ravel(),diabetes13predLasso.ravel()))
mean_sqr2 = mean_squared_error(diabetes13predLasso, y_test2)
print(mean_sqr2)
print()

print('Pearson Coefficient and MSE for Obesity Prediction in 2008 using Lasso')
print(pearsonr(np.asarray(y_test3).ravel(),obesity8predLasso.ravel()))
mean_sqr3 = mean_squared_error(obesity8predLasso, y_test3)
print(mean_sqr3)
print()

print('Pearson Coefficient and MSE for Obesity Prediction in 2013 using Lasso')
print(pearsonr(np.asarray(y_test4).ravel(),obesity13predLasso.ravel()))
mean_sqr4 = mean_squared_error(obesity13predLasso, y_test4)
print(mean_sqr4)

Pearson Coefficient and MSE for Diabetes Prediction in 2008 using Lasso
(0.8676361748253112, 5.360065358017706e-97)
1.2477833764971789

Pearson Coefficient and MSE for Diabetes Prediction in 2013 using Lasso
(0.8347762812150441, 4.080754834495669e-83)
1.9602930819082411

Pearson Coefficient and MSE for Obesity Prediction in 2008 using Lasso
(0.8193138674070828, 1.3303836671009461e-77)
4.826683153102001

Pearson Coefficient and MSE for Obesity Prediction in 2013 using Lasso
(0.8034015374594011, 1.8702031812770628e-72)
7.511292320821416


In [25]:
from sklearn.neighbors import KNeighborsRegressor
neigh1 = KNeighborsRegressor(n_neighbors=4)
neigh2 = KNeighborsRegressor(n_neighbors=4)
neigh3 = KNeighborsRegressor(n_neighbors=4)
neigh4 = KNeighborsRegressor(n_neighbors=4)

neigh1.fit(X_train,y_train)
neigh2.fit(X_train,y_train)
neigh3.fit(X_train,y_train)
neigh4.fit(X_train,y_train)

#test portion 
diabetes8predNeigh = neigh1.predict(X_test)
diabetes13predNeigh = neigh2.predict(X_test2)
obesity8predNeigh = neigh3.predict(X_test3)
obesity13predNeigh = neigh4.predict(X_test4)

print('Pearson Coefficient and MSE for Diabetes Prediction in 2008 using KNeighborsRegressor')
print(pearsonr(np.asarray(y_test).ravel(),diabetes8predNeigh.ravel()))
mean_sqr1 = mean_squared_error(diabetes8predNeigh, y_test)
print(mean_sqr1)
print()

print('Pearson Coefficient and MSE for Diabetes Prediction in 2013 using KNeighborsRegressor')
print(pearsonr(np.asarray(y_test2).ravel(),diabetes13predNeigh.ravel()))
mean_sqr2 = mean_squared_error(diabetes13predNeigh, y_test2)
print(mean_sqr2)
print()

print('Pearson Coefficient and MSE for Obesity Prediction in 2008 using KNeighborsRegressor')
print(pearsonr(np.asarray(y_test3).ravel(),obesity8predNeigh.ravel()))
mean_sqr3 = mean_squared_error(obesity8predNeigh, y_test3)
print(mean_sqr3)
print()

print('Pearson Coefficient and MSE for Obesity Prediction in 2013 using KNeighborsRegressor')
print(pearsonr(np.asarray(y_test4).ravel(),obesity13predNeigh.ravel()))
mean_sqr4 = mean_squared_error(obesity13predNeigh, y_test4)
print(mean_sqr4)

Pearson Coefficient and MSE for Diabetes Prediction in 2008 using KNeighborsRegressor
(0.8718923837497738, 4.5746859307970246e-99)
1.1334220437134102

Pearson Coefficient and MSE for Diabetes Prediction in 2013 using KNeighborsRegressor
(0.8120853717853946, 3.3365829937162197e-75)
3.8694981960068424

Pearson Coefficient and MSE for Obesity Prediction in 2008 using KNeighborsRegressor
(0.7059801194177087, 8.100849464115205e-49)
369.81290714683126

Pearson Coefficient and MSE for Obesity Prediction in 2013 using KNeighborsRegressor
(0.6533610011889457, 1.005068931545969e-39)
461.25043186116613


In [22]:
from sklearn.neural_network import MLPRegressor

In [27]:
NN1 = MLPRegressor()
NN2 = MLPRegressor()
NN3 = MLPRegressor()
NN4 = MLPRegressor()

NN1.fit(X_train,y_train)
NN2.fit(X_train,y_train)
NN3.fit(X_train,y_train)
NN4.fit(X_train,y_train)

#test portion 
diabetes8predNN = NN1.predict(X_test)
diabetes13predNN = NN2.predict(X_test2)
obesity8predNN = NN3.predict(X_test3)
obesity13predNN = NN4.predict(X_test4)

print('Pearson Coefficient and MSE for Diabetes Prediction in 2008 using MLPRegressor')
print(pearsonr(np.asarray(y_test).ravel(),diabetes8predNN.ravel()))
mean_sqr1 = mean_squared_error(diabetes8predNN, y_test)
print(mean_sqr1)
print()

print('Pearson Coefficient and MSE for Diabetes Prediction in 2013 using MLPRegressor')
print(pearsonr(np.asarray(y_test2).ravel(),diabetes13predNN.ravel()))
mean_sqr2 = mean_squared_error(diabetes13predNN, y_test2)
print(mean_sqr2)
print()

print('Pearson Coefficient and MSE for Obesity Prediction in 2008 using MLPRegressor')
print(pearsonr(np.asarray(y_test3).ravel(),obesity8predNN.ravel()))
mean_sqr3 = mean_squared_error(obesity8predNN, y_test3)
print(mean_sqr3)
print()

print('Pearson Coefficient and MSE for Obesity Prediction in 2013 using MLPRegressor')
print(pearsonr(np.asarray(y_test4).ravel(),obesity13predNN.ravel()))
mean_sqr4 = mean_squared_error(obesity13predNN, y_test4)
print(mean_sqr4)

Pearson Coefficient and MSE for Diabetes Prediction in 2008 using MLPRegressor
(0.8475054356052274, 4.218064790764552e-88)
1.425592221294647

Pearson Coefficient and MSE for Diabetes Prediction in 2013 using MLPRegressor
(0.7620428017839961, 4.948758793914017e-61)
4.487663790085343

Pearson Coefficient and MSE for Obesity Prediction in 2008 using MLPRegressor
(0.7031067536019321, 2.8622228077965127e-48)
370.49666506132905

Pearson Coefficient and MSE for Obesity Prediction in 2013 using MLPRegressor
(0.5930189087557544, 2.6557646942945248e-31)
461.19601038605776


In [33]:
#now drop additional health features to see how it influences results
df8 = df8.drop('PCT_HSPA15', 1)
df8 = df8.drop('RECFAC09', 1)
df8 = df8.drop('RECFAC14', 1)
df8 = df8.drop('PCH_RECFAC_09_14', 1)
df8 = df8.drop('RECFACPTH09', 1)
df8 = df8.drop('RECFACPTH14', 1)
df8 = df8.drop('PCH_RECFACPTH_09_14', 1)

datawithouthealth = [df1,df2,df3,df4,df5,df6,df7,df8,df9] 

#formatting all instances
X = []
i = 0
for i in range(0,3143):
    current_row = []
    for frame in datawithouthealth:
        mat = frame.as_matrix()
        current_row.append(mat[i][3:])
    X.append(list(itertools.chain(*current_row)))

#some features are missing data for specific instances, so I want to find them and deal with them
missing = np.argwhere(np.isnan(X))

feats = []
for i in range(0,len(missing)):
    feats.append(missing[i][1])

#these are two of the ~10 categorical features. Only these two have missing features
#since they are 0 or 1 binary classes, it doesn't make sense to replace NaN values with the mean of the column
#so instead I replace it will -1 as a "missing" class
for i in range(0,len(missing)):
    if missing[i][1] == 249:
        X[missing[i][0]][249] = -1
    elif missing[i][1] == 250:
        X[missing[i][0]][250] = -1

#all other features are continuous so I can impute NaN values with the mean of the feature!
from sklearn.preprocessing import Imputer
# missing_values is the value of your placeholder, strategy is if you'd like mean, median or mode, and axis=0 means it calculates the imputation based on the other feature values for that sample
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(X)
inputspace = imp.transform(X)

#I didn't previously realize that the diabetes and obesity vectors had Nan 
#now fixing it similarly with the below cells, only a few were missing (3 or 4)
imp2 = Imputer(missing_values='NaN', strategy='mean', axis=0)
diabetes08=diabetes08.reshape(-1,1)
imp2.fit(diabetes08)
outputspace1 = imp2.transform(diabetes08)

imp3 = Imputer(missing_values='NaN', strategy='mean', axis=0)
diabetes13=diabetes13.reshape(-1,1)
imp3.fit(diabetes13)
outputspace2 = imp3.transform(diabetes13)

imp4 = Imputer(missing_values='NaN', strategy='mean', axis=0)
obesity08=obesity08.reshape(-1,1)
imp4.fit(obesity08)
outputspace3 = imp4.transform(obesity08)

imp5 = Imputer(missing_values='NaN', strategy='mean', axis=0)
obesity13=obesity13.reshape(-1,1)
imp5.fit(obesity13)
outputspace4 = imp5.transform(obesity13)

inputspacewithouthealth = prep.scale(inputspace)

X_train, X_test, y_train, y_test = train_test_split(inputspacewithouthealth,outputspace1.ravel(), test_size=0.1, random_state=42)

X_train2, X_test2, y_train2, y_test2 = train_test_split(inputspacewithouthealth,outputspace2.ravel(), test_size=0.1, random_state=42)

X_train3, X_test3, y_train3, y_test3 = train_test_split(inputspacewithouthealth,outputspace3.ravel(), test_size=0.1, random_state=42)

X_train4, X_test4, y_train4, y_test4 = train_test_split(inputspacewithouthealth,outputspace4.ravel(), test_size=0.1, random_state=42)



{1, 2, 4, 6, 7, 9, 11, 12, 14, 15, 16, 18, 19, 21, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 43, 46, 49, 52, 55, 58, 61, 64, 67, 69, 70, 73, 76, 79, 82, 85, 88, 93, 94, 95, 99, 100, 101, 115, 116, 117, 118, 125, 126, 127, 128, 129, 130, 146, 147, 148, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 169, 170, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 259, 260, 262, 265}


In [36]:
diab8clfSVR = SVR(C=1.0, epsilon=0.2)
diab13clfSVR = SVR(C=1.0, epsilon=0.2)
obes8clfSVR = SVR(C=1.0, epsilon=0.2)
obes13clfSVR = SVR(C=1.0, epsilon=0.2)

diab8clfSVR.fit(X_train,y_train)
diab13clfSVR.fit(X_train2,y_train2)
obes8clfSVR.fit(X_train3,y_train3)
obes13clfSVR.fit(X_train4,y_train4)

diabetes8predSVR = diab8clfSVR.predict(X_test)
diabetes13predSVR = diab13clfSVR.predict(X_test2)
obesity8predSVR = obes8clfSVR.predict(X_test3)
obesity13predSVR = obes13clfSVR.predict(X_test4)

print('Pearson Coefficient and MSE for Diabetes Prediction in 2008 using SVR without health factors')
print(pearsonr(np.asarray(y_test).ravel(),diabetes8predSVR.ravel()))
mean_sqr1 = mean_squared_error(diabetes8predSVR, y_test)
print(mean_sqr1)
print()

print('Pearson Coefficient and MSE for Diabetes Prediction in 2013 using SVR without health factors')
print(pearsonr(np.asarray(y_test2).ravel(),diabetes13predSVR.ravel()))
mean_sqr2 = mean_squared_error(diabetes13predSVR, y_test2)
print(mean_sqr2)
print()

print('Pearson Coefficient and MSE for Obesity Prediction in 2008 using SVR without health factors')
print(pearsonr(np.asarray(y_test3).ravel(),obesity8predSVR.ravel()))
mean_sqr3 = mean_squared_error(obesity8predSVR, y_test3)
print(mean_sqr3)
print()

print('Pearson Coefficient and MSE for Obesity Prediction in 2013 using SVR without health factors')
print(pearsonr(np.asarray(y_test4).ravel(),obesity13predSVR.ravel()))
mean_sqr4 = mean_squared_error(obesity13predSVR, y_test4)
print(mean_sqr4)

Pearson Coefficient and MSE for Diabetes Prediction in 2008 using SVR without health factors
(0.9113778317864002, 1.0350401063976097e-122)
0.8271951488977475

Pearson Coefficient and MSE for Diabetes Prediction in 2013 using SVR without health factors
(0.8745309419363685, 2.1892767613936128e-100)
1.5006980490061568

Pearson Coefficient and MSE for Obesity Prediction in 2008 using SVR without health factors
(0.8346926714870706, 4.3860305140648884e-83)
4.579762587043926

Pearson Coefficient and MSE for Obesity Prediction in 2013 using SVR without health factors
(0.8158009769996849, 2.0099857908752087e-76)
7.172651385504216


In [38]:
reg1 = linear_model.Lasso(alpha = 0.1)
reg1.fit(X_train,y_train)

reg2 = linear_model.Lasso(alpha = 0.1)
reg2.fit(X_train2,y_train2)

reg3 = linear_model.Lasso(alpha = 0.1)
reg3.fit(X_train3,y_train3)

reg4 = linear_model.Lasso(alpha = 0.1)
reg4.fit(X_train4,y_train4)

#test portion 
diabetes8predLasso = reg1.predict(X_test)
diabetes13predLasso = reg2.predict(X_test2)
obesity8predLasso = reg3.predict(X_test3)
obesity13predLasso = reg4.predict(X_test4)

from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
print('Pearson Coefficient and MSE for Diabetes Prediction in 2008 using Lasso w/o health factors')
print(pearsonr(np.asarray(y_test).ravel(),diabetes8predLasso.ravel()))
mean_sqr1 = mean_squared_error(diabetes8predLasso, y_test)
print(mean_sqr1)
print()

print('Pearson Coefficient and MSE for Diabetes Prediction in 2013 using Lasso w/o health factors')
print(pearsonr(np.asarray(y_test2).ravel(),diabetes13predLasso.ravel()))
mean_sqr2 = mean_squared_error(diabetes13predLasso, y_test2)
print(mean_sqr2)
print()

print('Pearson Coefficient and MSE for Obesity Prediction in 2008 using Lasso w/o health factors')
print(pearsonr(np.asarray(y_test3).ravel(),obesity8predLasso.ravel()))
mean_sqr3 = mean_squared_error(obesity8predLasso, y_test3)
print(mean_sqr3)
print()

print('Pearson Coefficient and MSE for Obesity Prediction in 2013 using Lasso w/o health factors')
print(pearsonr(np.asarray(y_test4).ravel(),obesity13predLasso.ravel()))
mean_sqr4 = mean_squared_error(obesity13predLasso, y_test4)
print(mean_sqr4)

Pearson Coefficient and MSE for Diabetes Prediction in 2008 using Lasso w/o health factors
(0.8678542242497431, 4.216371106252921e-97)
1.246824158590935

Pearson Coefficient and MSE for Diabetes Prediction in 2013 using Lasso w/o health factors
(0.836666850110862, 7.89894441182933e-84)
1.9438202565555593

Pearson Coefficient and MSE for Obesity Prediction in 2008 using Lasso w/o health factors
(0.8178821880957337, 4.051746748525077e-77)
4.857236881597624

Pearson Coefficient and MSE for Obesity Prediction in 2013 using Lasso w/o health factors
(0.8016144642508652, 6.614373965213981e-72)
7.566092833073249


In [39]:
neigh1 = KNeighborsRegressor(n_neighbors=4)
neigh2 = KNeighborsRegressor(n_neighbors=4)
neigh3 = KNeighborsRegressor(n_neighbors=4)
neigh4 = KNeighborsRegressor(n_neighbors=4)

neigh1.fit(X_train,y_train)
neigh2.fit(X_train,y_train)
neigh3.fit(X_train,y_train)
neigh4.fit(X_train,y_train)

#test portion 
diabetes8predNeigh = neigh1.predict(X_test)
diabetes13predNeigh = neigh2.predict(X_test2)
obesity8predNeigh = neigh3.predict(X_test3)
obesity13predNeigh = neigh4.predict(X_test4)

print('Pearson Coefficient and MSE for Diabetes Prediction in 2008 using KNeighborsRegressor w/o health factors')
print(pearsonr(np.asarray(y_test).ravel(),diabetes8predNeigh.ravel()))
mean_sqr1 = mean_squared_error(diabetes8predNeigh, y_test)
print(mean_sqr1)
print()

print('Pearson Coefficient and MSE for Diabetes Prediction in 2013 using KNeighborsRegressor w/o health factors')
print(pearsonr(np.asarray(y_test2).ravel(),diabetes13predNeigh.ravel()))
mean_sqr2 = mean_squared_error(diabetes13predNeigh, y_test2)
print(mean_sqr2)
print()

print('Pearson Coefficient and MSE for Obesity Prediction in 2008 using KNeighborsRegressor w/o health factors')
print(pearsonr(np.asarray(y_test3).ravel(),obesity8predNeigh.ravel()))
mean_sqr3 = mean_squared_error(obesity8predNeigh, y_test3)
print(mean_sqr3)
print()

print('Pearson Coefficient and MSE for Obesity Prediction in 2013 using KNeighborsRegressor w/o health factors')
print(pearsonr(np.asarray(y_test4).ravel(),obesity13predNeigh.ravel()))
mean_sqr4 = mean_squared_error(obesity13predNeigh, y_test4)
print(mean_sqr4)

Pearson Coefficient and MSE for Diabetes Prediction in 2008 using KNeighborsRegressor w/o health factors
(0.8731266976240107, 1.113058041341443e-99)
1.123170059586426

Pearson Coefficient and MSE for Diabetes Prediction in 2013 using KNeighborsRegressor w/o health factors
(0.8110922724567246, 6.995500679693879e-75)
3.8765636721973187

Pearson Coefficient and MSE for Obesity Prediction in 2008 using KNeighborsRegressor w/o health factors
(0.7063068062137856, 7.011093514335402e-49)
369.7592742103233

Pearson Coefficient and MSE for Obesity Prediction in 2013 using KNeighborsRegressor w/o health factors
(0.6464000403404199, 1.178595510133878e-38)
461.31995765481696


In [40]:
NN1 = MLPRegressor()
NN2 = MLPRegressor()
NN3 = MLPRegressor()
NN4 = MLPRegressor()

NN1.fit(X_train,y_train)
NN2.fit(X_train,y_train)
NN3.fit(X_train,y_train)
NN4.fit(X_train,y_train)

#test portion 
diabetes8predNN = NN1.predict(X_test)
diabetes13predNN = NN2.predict(X_test2)
obesity8predNN = NN3.predict(X_test3)
obesity13predNN = NN4.predict(X_test4)

print('Pearson Coefficient and MSE for Diabetes Prediction in 2008 using MLPRegressor w/o health factors')
print(pearsonr(np.asarray(y_test).ravel(),diabetes8predNN.ravel()))
mean_sqr1 = mean_squared_error(diabetes8predNN, y_test)
print(mean_sqr1)
print()

print('Pearson Coefficient and MSE for Diabetes Prediction in 2013 using MLPRegressor w/o health factors')
print(pearsonr(np.asarray(y_test2).ravel(),diabetes13predNN.ravel()))
mean_sqr2 = mean_squared_error(diabetes13predNN, y_test2)
print(mean_sqr2)
print()

print('Pearson Coefficient and MSE for Obesity Prediction in 2008 using MLPRegressor w/o health factors')
print(pearsonr(np.asarray(y_test3).ravel(),obesity8predNN.ravel()))
mean_sqr3 = mean_squared_error(obesity8predNN, y_test3)
print(mean_sqr3)
print()

print('Pearson Coefficient and MSE for Obesity Prediction in 2013 using MLPRegressor w/o health factors')
print(pearsonr(np.asarray(y_test4).ravel(),obesity13predNN.ravel()))
mean_sqr4 = mean_squared_error(obesity13predNN, y_test4)
print(mean_sqr4)

Pearson Coefficient and MSE for Diabetes Prediction in 2008 using MLPRegressor w/o health factors
(0.8228820465734332, 7.936681130425816e-79)
1.7801359177847293

Pearson Coefficient and MSE for Diabetes Prediction in 2013 using MLPRegressor w/o health factors
(0.7844537162220838, 6.577610612573506e-67)
4.362514049845348

Pearson Coefficient and MSE for Obesity Prediction in 2008 using MLPRegressor w/o health factors
(0.661047688988831, 6.142132708604347e-41)
365.82789012566735

Pearson Coefficient and MSE for Obesity Prediction in 2013 using MLPRegressor w/o health factors
(0.6360912400049792, 4.0225403613473576e-37)
459.06288515818636
